In [1]:
import torch

%load_ext autoreload
%autoreload 1
%aimport utils_data
%aimport utils_model
%aimport utils_attack

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Datasets and dataloaders preparation 

In [3]:
# (train_set, test_set), (train_loader, test_loader) = utils_data.get_train_test_dataloaders(
#     dataset_type='cifar10', 
#     root_data_folder='./data', 
#     batch_size=64
# )

Because of the problem with the cell above for CIFAR10 use the cell below

In [4]:
import os
import random
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

'''source https://github.com/kuangliu/pytorch-cifar/blob/master/main.py'''
train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

data_folder = '/home/Skoltech_MSc/ML/HW2_v1.0/data'

train_set = torchvision.datasets.CIFAR10(
    root=data_folder, train=True, 
    download=False, transform=train_transform
)
test_set = torchvision.datasets.CIFAR10(
    root=data_folder, train=False,
    download=False, transform=test_transform
)

batch_size = 64

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size,
    shuffle=True, num_workers=2
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size,
    shuffle=False, num_workers=2
)

# Model

## Loading and fine-tuning ResNet50 pretrained on ImageNet 
**(you can skip this part if you already have a model pretrained on CIFAR10/100)**

In [5]:
model = torchvision.models.resnet50(pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model = model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
utils_data.set_random_seeds(seed_value=123, device=device)

'''source https://github.com/kuangliu/pytorch-cifar/blob/master/main.py'''
optimizer = optim.SGD(model.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

criterion = nn.CrossEntropyLoss()

In [7]:
model = utils_model.train(train_loader, test_loader, model, criterion, optimizer, scheduler, n_epochs=10, device=device)

 10%|█         | 1/10 [01:31<13:44, 91.65s/it]

[Epoch 1] train loss: 1.114; train acc: 0.61; test loss: 0.723; test acc: 0.75


 20%|██        | 2/10 [02:59<11:53, 89.18s/it]

[Epoch 2] train loss: 0.699; train acc: 0.76; test loss: 0.592; test acc: 0.80


 30%|███       | 3/10 [04:31<10:35, 90.80s/it]

...

[Epoch 8] train loss: 0.357; train acc: 0.88; test loss: 0.416; test acc: 0.86


 90%|█████████ | 9/10 [13:29<01:29, 89.58s/it]

[Epoch 9] train loss: 0.335; train acc: 0.88; test loss: 0.403; test acc: 0.86


100%|██████████| 10/10 [14:57<00:00, 89.74s/it]

[Epoch 10] train loss: 0.315; train acc: 0.89; test loss: 0.400; test acc: 0.87


In [15]:
from sklearn.metrics import accuracy_score

In [20]:
all_preds, all_labels = torch.tensor([], device=device), torch.tensor([], device=device)

for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)
    pred = model(images)
    all_preds = torch.cat([all_preds, pred.argmax(axis=1)])
    all_labels = torch.cat([all_labels, labels])
    
accuracy_score(all_labels.detach().cpu().numpy(), all_preds.detach().cpu().numpy())

100%|██████████| 157/157 [00:04<00:00, 33.34it/s]


0.8699

In [21]:
path_to_model_save = './models'
torch.save(model.state_dict(), os.path.join(path_to_model_save, 'cifar10'))

## Load a model pretrained on CIFAR10/100
**to load pretrained model uncomment the cell below**

In [5]:
path_to_model_save = './models'
model = torchvision.models.resnet50(pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
model.load_state_dict(
    torch.load(
        os.path.join(path_to_model_save, 'cifar10')
    )
)
model = model.to(device)

In [6]:
utils_data.set_random_seeds(seed_value=123, device=device)

'''source https://github.com/kuangliu/pytorch-cifar/blob/master/main.py'''
optimizer = optim.SGD(model.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

criterion = nn.CrossEntropyLoss()

# FGSM attack

In [7]:
# targets, total_preds, ori_preds, adv_samples, adv_targets, true_samples, true_targets = utils_attack.test_on_adv(
#     model,
#     test_loader,
#     criterion,
#     params={"eps": 1e-3},
#     method="fgsm",
#     n_samples_ret=5,
#     device=device
# )

In [8]:
# targets.shape, total_preds.shape, ori_preds.shape, adv_samples.shape, adv_targets.shape, true_samples.shape, true_targets.shape

In [9]:
results_df = utils_attack.test_robustness_simple(
    model,
    test_loader,
    criterion,
    (1e-3, 1e0, 20), # (1e-3, 1e0, 20),
    attack_type="fgsm",
    n_samples_ret=5,
    device=device
)
results_df

100%|██████████| 20/20 [08:05<00:00, 24.28s/it]


Metric,Accuracy
Noise sigma,
0.001000,0.8541
0.001438,0.8481
0.002069,0.8376
0.002976,0.8220
0.004281,0.8004
0.006158,0.7697
0.008859,0.7279
0.012743,0.6660
0.018330,0.5863
